In [1]:
import pandas as pd
import numpy as np
import vnstock as vn

In [9]:
transactions_df = pd.DataFrame({
    'Date': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04'],
    'Stock': ['AAPL', 'GOOGL', 'MSFT', 'AAPL'],
    'Action': ['Buy', 'Buy', 'Sell', 'Sell'],
    'Quantity': [100,200,500,300],
    'Price': [150, 2500, 200, 160]
})

recent_prices_df = pd.DataFrame({
    'Stock': ['AAPL', 'GOOGL', 'MSFT'],
    'Recent Price': [155, 2600, 210]
})

portfolio_df = pd.DataFrame({
    'Stock': ['AAPL', 'GOOGL'],
    'Quantity': [100, 200]
})

In [3]:
transactions_df

,Date,Stock,Action,Quantity,Price
0,2024-01-01,AAPL,Buy,100,150
1,2024-01-02,GOOGL,Buy,200,2500
2,2024-01-03,MSFT,Sell,500,200
3,2024-01-04,AAPL,Sell,300,160


In [10]:
recent_prices_df

,Stock,Recent Price
0,AAPL,155
1,GOOGL,2600
2,MSFT,210


In [5]:
portfolio_df

,Stock,Quantity
0,AAPL,100
1,GOOGL,200


In [46]:
def get_recent_price(stock_name):
    recent_price = recent_prices_df.loc[recent_prices_df['Stock'] == stock_name, 'Price'].values[0]
    return recent_price

def calculate_daily_returns(transactions_df, recent_prices_df):
    merged_df = pd.merge(transactions_df, recent_prices_df, on='Stock', how='left')
    merged_df.sort_values(by='Date', inplace=True)
    merged_df['Daily Return'] = merged_df.groupby('Stock')['Price'].pct_change()
    merged_df.dropna(subset=['Daily Return'], inplace=True)
    daily_returns_df = merged_df[['Date', 'Stock', 'Daily Return']]
    return daily_returns_df

def cal_portfolio_daily_return(transactions_df,recent_prices_df, portfolio_df):
    daily_returns_df = calculate_daily_returns(transactions_df, recent_prices_df)
    merged_df = pd.merge(daily_returns_df, portfolio_df, on='Stock', how='inner')
    merged_df['Weighted Return'] = merged_df['Daily Return'] * merged_df['Quantity']
    total_weighted_return = merged_df['Weighted Return'].sum()
    total_portfolio_quantity = merged_df['Quantity'].sum()
    mean_return = total_weighted_return / total_portfolio_quantity
    portfolio_daily_return = merged_df.groupby('Date')['Weighted Return'].sum().reset_index()

    return merged_df, mean_return, portfolio_daily_return

def cal_port_current_value(portfolio_df, current_prices):
    initial_value = 123

    for asset, quantity in portfolio_df.items():
        price = current_prices.get(asset, 0)
        asset_value = price*quantity
        current_value += asset_value
    return current_value

def record_action(signal_df, portfolio_df, recent_prices_df):
    transactions = []

    for index, signal in signal_df.iterrows():
        date = signal['Date']
        stock = signal['Stock']
        action = signal['Action']
        quantity = signal['Quantity']
        price = signal['Price']
        # or price will be determine if the algorith is applied in real market
        # price = recent_prices_df.loc[recent_prices_df['Stock'] == stock, 'Price'].values[0]

        if action == 'Buy':
            transaction = {
                'Date': date,
                'Stock': stock,
                'Action': action,
                'Quantity': quantity,
                'Price': price
            }
            transactions.append(transaction)

            if stock in portfolio_df['Stock'].values:
                #Update existing stock quantity
                portfolio_df.loc[portfolio_df['Stock'] == stock, 'Quantity'] += quantity
            else:
                #Add new stock to the portfolio
                portfolio_df = portfolio_df.append({'Stock': stock, 'Quantity': quantity, 'Price': price}, ignore_index = True)
        
        elif action == 'Sell':
            if stock in portfolio_df['Stock'].values:
                transaction = {
                    'Date': date,
                    'Stock': stock,
                    'Action': action,
                    'Quantity': quantity,
                    'Price': price
            }
            transactions.append(transaction)

            #Reduce stock quantity
            portfolio_df.loc[portfolio_df['Stock']== stock, 'Quantity']-= quantity
            #Remove stock if quantity becomes zero
            if portfolio_df.loc[portfolio_df['Stock'] == stock, 'Quantity'].values[0] <= 0:
                portfolio_df = portfolio_df[portfolio_df['Stock'] != stock]
        
        transaction_df = pd.DataFrame(transactions)
        return transaction_df


In [21]:
portfolio_daily_return_df = cal_portfolio_daily_return(transactions_df, recent_prices_df, portfolio_df)
portfolio_daily_return_df.head()

mean_return = cal_portfolio_daily_return(transactions_df, recent_prices_df, portfolio_df,return_dataframe=False)
print(mean_return)

0.06666666666666665


In [26]:
daily_return, mean_return, portfolio_daily_return_df = cal_portfolio_daily_return(transactions_df, recent_prices_df, portfolio_df)


In [27]:
print(portfolio_daily_return_df)

         Date  Weighted Return
0  2024-01-04         6.666667


In [28]:
daily_return

,Date,Stock,Daily Return,Quantity,Weighted Return
0,2024-01-04,AAPL,0.066667,100,6.666667


In [29]:
mean_return

0.06666666666666665

In [13]:
daily_returns_df = calculate_daily_returns(transactions_df, recent_prices_df)
print("Daily Returns:")
print(daily_returns_df)

Daily Returns:
         Date Stock  Daily Return
3  2024-01-04  AAPL      0.066667


In [35]:
record_action_df = record_action(portfolio_df,transactions_df)
record_action_df

,Stock,Quantity
0,AAPL,200
1,GOOGL,200


In [48]:
signal_df = pd.DataFrame({
    'Date': ['2024-01-01', '2024-01-02', '2024-01-03'],
    'Stock': ['AAPL', 'GOOGL', 'AAPL'],
    'Action': ['Buy', 'Sell', 'Buy'],
    'Quantity': [100, 200, 150],
    'Price':[150,200,160]
})

In [49]:
signal_df.head()

,Date,Stock,Action,Quantity,Price
0,2024-01-01,AAPL,Buy,100,150
1,2024-01-02,GOOGL,Sell,200,200
2,2024-01-03,AAPL,Buy,150,160


In [50]:
transaction_df = record_action(signal_df, portfolio_df, recent_prices_df)
transaction_df

,Date,Stock,Action,Quantity,Price
0,2024-01-01,AAPL,Buy,100,150
